In [ ]:
# Initial imports and notebook setup, click arrow to show
import sys 
import os

from HARK.ConsumptionSaving.ConsIndShockModel import *
from HARK.utilities import plotFuncsDer, plotFuncs

This module contains models to solve canonical consumption-saving models with idiosyncratic shocks to income.  All models here assume CRRA utility with geometric discounting, no bequest motive, and income shocks are fully transitory or fully permanent.

$\texttt{ConsIndShockModel}$ currently solves three types of models:
1. A very basic "perfect foresight" consumption-saving model with no uncertainty.
2. A consumption-saving model with risk over transitory and permanent income shocks.
3. The model described in (2), with an interest rate for debt that differs from the interest rate for savings.

See [NARK](https://github.com/econ-ark/NARK) for information on variable naming conventions.
See [HARK documentation](https://github.com/econ-ark/HARK/Documentation) for brief mathematical descriptions of the models being solved.  Detailed mathematical references are referenced _in situ_ below.

## Perfect Foresight Consumer

This model represents a consumer who has no uncertainty other than mortality. His problem is defined by a coefficient of relative risk aversion, an intertemporal discount factor, interest factor, and time sequences of the permanent income growth rate and survival probability.

The solution of this model is described in details here [PerfForesightCRRA](http://econ.jhu.edu/people/ccarroll/public/lecturenotes/consumption/PerfForesightCRRA)


The agent's problem can be written in Bellman form as:

\begin{eqnarray*}
V_t(M_t) &=& \max_{C_t} U(C_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [V_{t+1}(M_{t+1}) ], \\
A_t &=& M_t - C_t, \\
M_{t+1} &=& R A_t + Y_{t+1}, \\
Y_{t+1} &=& \Gamma_{t+1}Y_{t+1}, \\
U(C) &=& \frac{C^{1-\rho}}{1-\rho}.
\end{eqnarray*}

The one period problem for this model is solved by the function $\texttt{solveConsPerfForesight}$, which creates an instance of the class $\texttt{ConsPerfForesightSolver}$. The class $\texttt{PerfForesightConsumerType}$ extends $\texttt{AgentType}$ to represents agents in this model. To construct an instance of this class, several parameters must be passed to the constructor as shown in the table below. 

Parameters can be either "primitive" if they are directly specified by the user or "constructed" if they are built by a class method using simple parameters specified by the user. 

### Example parameter values to solve an instance of PerfForesightConsumerType

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
| $\beta$ |Intertemporal discount factor  | $\texttt{DiscFac}$ | 0.96 | |
| $\rho $ |Coefficient of relative risk aversion | $\texttt{CRRA}$ | 2.0 | |
| $R $ | Risk free interest factor | $\texttt{Rfree}$ | 1.03 | |
| $1 - \mathsf{D}$ |Survival probability | $\texttt{LivPrb}$ | [0.98] | |
|$\Gamma$|Permanent income growth factor|$\texttt{PermGroFac}$|[1.01]||
|(none)|Number of agents of this type|$\texttt{AgentCount}$|10000|||

In [ ]:
# This cell defines a dictionary to make an instance of a "perfect foresight" consumer.
PerfForesightDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : [0.98],                     # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.01],                  # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
}

In [ ]:
# Make and solve an example of a "perfect foresight" consumer
PFexample = PerfForesightConsumerType(**PerfForesightDict)   
PFexample.cycles = 0 # Make this type have an infinite horizon
PFexample.solve()

After this model has been solved, the consumption functions reside in the attribute $\texttt{cFunc}$ of each element of $\texttt{ConsumerType.solution}$.  This method creates a (time varying) attribute $\texttt{cFunc}$ that contains a list of consumption functions.

In [ ]:
# Unpack the consumption functions into their own field for easier access
PFexample.unpackcFunc()

In [ ]:
# Plot the perfect foresight consumption function
print('Linear consumption function:')
mMin = PFexample.solution[0].mNrmMin
plotFuncs(PFexample.cFunc[0],mMin,mMin+10)

In [ ]:
# Simulate some data
PFexample.timeFwd()
PFexample.T_sim = 120 # Set number of simulation periods
PFexample.track_vars = ['mNrmNow']
PFexample.initializeSim()
PFexample.simulate()

## Consumer with idiosyncratic income shocks

This model represents a consumer with idiosyncratic shocks to her/his permanent and transitory income. Most specifically, the agent foresses that s/he will experience shocks to income that are fully transitory or fully permanent. That is, this agent is identical to the perfect foresight agent except that income is subsject to permanent, $\psi$, and transitory shocks, $\theta$, and s/he might have an artificial borrowing constraint $\underline{a}$.


Solve a model like the one analyzed in [BufferStockTheory](http://econ.jhu.edu/people/ccarroll/papers/BufferStockTheory/)

The agent's problem can be written in (normalized) Bellman form as:

\begin{eqnarray*}
v_t(m_t) &=& \max_{c_t} u(c_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [(\Gamma_{t+1}\psi_{t+1})^{1-\rho} v_{t+1}(m_{t+1}) ], \\
a_t &=& m_t - c_t, \\
a_t &\geq& \underline{a}, \\
m_{t+1} &=& R/(\Gamma_{t+1}\psi_{t+1}) a_t + \theta_{t+1}, \\
\psi_{t},\theta_{t} \sim F_{t}, &\qquad& \mathbb{E} [F_{\psi t}] = 1, \\
u(c) &=& \frac{c^{1-\rho}}{1-\rho}
\end{eqnarray*}


In [ ]:
# This cell defines a dictionary to make an instance of a "idiosyncratic shock" consumer.
IdiosyncDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : [0.98],                     # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.01],                  # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [None],                   # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
    
    # Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1],                  # Standard deviation of log permanent shocks to income
    "TranShkStd" : [0.1],                  # Standard deviation of log transitory shocks to income
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    
    # A few other paramaters
    "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                   # Preference shocks currently only compatible with linear cFunc
    "vFuncBool" : False,                    # Whether to calculate the value function during solution          
}

In [ ]:
# Make an example of an idiosyncratic income shock consumer
IndShockExample = IndShockConsumerType(**IdiosyncDict)
IndShockExample.cycles = 0 # Make this type have an infinite horizon

The one period problem for this model is solved by the function $\texttt{solveConsIndShock}$, which creates an instance of the class $\texttt{ConsIndShockSolver}$. The class $\texttt{IndShockConsumerType}$ extends $\texttt{PerfForesightConsumerType}$ to represents agents in this model. To construct an instance of this class, several parameters must be passed to the constructor as shown in the table below. 

Note that most of these parameters are *indirect* inputs to the consumer's model - they are used to construct direct inputs to the one period problem.

### Example parameter values to solve an instance of IndShockConsumerType

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
| $ (none) $ | Grid of "assets above minimum" | $\texttt{aXtraGrid}$ | - |$\surd$ |
|$F $ |A list containing three arrays of floats, representing a discrete <br> approximation to the income process: <br>event probabilities, permanent shocks, transitory shocks | $\texttt{IncomeDstn}$ | - |$\surd$ |
| $\sigma_\theta $ | Minimum of "assets above minimum" grid | $\texttt{TranShkStd}$ | - |$\surd$ |
| $ \sigma_\psi $ | Minimum of "assets above minimum" grid | $\texttt{PermShkStd}$ | - |$\surd$ |
| $\underline{a} $ |Artificial borrowing constraint, as asset-to-persistent income ratio | $\texttt{BoroCnstArt}$ | 0.0 | | 
| $(none) $ |Indicator of whether $\texttt{vFunc}$ should be computed | $\texttt{vFuncBool}$ | 'True' | |
| $(none)$ |Indicator of whether $\texttt{cFunc}$ should use cubic lines | $\texttt{CubicBool}$ | 'False' |  |

### Constructed inputs to solve IndShockExample

To construct the "assets above minimum grid", $\texttt{aXtraGrid}$, the following attributes can be used and changed easily as needed:

| Param | Description | Type | Value|
| :---: | --- | --- | --- | :---: |
| $\texttt{aXtraMin}$ | Minimum of "assets above minimum" grid | float | 0.001  | 
| $\texttt{aXtraMax}$ | Maximum of "assets above minimum" grid | int | 20  | 
| $\texttt{aXtraCount}$  | Number of points in "assets above minimum" grid | int| 48 | 
| $\texttt{aXtraNestFac}$  | Exponential nesting factor when constructing "assets above minimum" grid | int | 3 | 

To construct the income process, ($F_{\psi t}$, $F_{\theta t}$), the following attributes can be used and changed easily as needed:

| Param | Description | Type | Value|
| :---: | --- | --- | --- | :---: |
| $\texttt{TranShkCount}$  | Number of discrete values in transitory shock distribution, $ N_{\theta} $ | int | 7 | 
| $\texttt{PermShkCount}$ | Number of discrete values in permanent shock distribution, $ N_{\psi} $ | int | 7 | 
| $\texttt{PermShkStd}$ | List of standard deviations in log permanent income shocks  | [float]  | [0.1] | 
| $\texttt{TranShkStd}$ |  List of standard deviations in log transitory income shocks | [float]  | [0.1] | 
| $\texttt{UnempPrb}$ | Probability of unemployment while working| float | 0.05  | 
| $\texttt{UnempPrbRet}$ | Probability of "unemployment" while retired | float | 0.005   | 
| $\texttt{IncUnemp}$ | Unemployment benefits replacement rate | float | 0.3  | 
| $\texttt{IncUnempRet}$ | "Unemployment" benefits when retired | float | 0.0  | 

In [ ]:
# Solve the model and unpack the consumption functions
IndShockExample.solve()
IndShockExample.unpackcFunc()

In [ ]:
# Plot the consumption function and MPC for the infinite horizon consumer
print('Concave consumption function:')
plotFuncs(IndShockExample.cFunc[0],IndShockExample.solution[0].mNrmMin,5)
print('Marginal propensity to consume function:')
plotFuncsDer(IndShockExample.cFunc[0],IndShockExample.solution[0].mNrmMin,5)

In [ ]:
# Compare the consumption functions for the perfect foresight and idiosyncratic shock types. 
# Risky income cFunc asymptotically approaches perfect foresight cFunc.
print('Consumption functions for perfect foresight vs idiosyncratic shocks:')            
plotFuncs([PFexample.cFunc[0],IndShockExample.cFunc[0]],IndShockExample.solution[0].mNrmMin,100)

In [ ]:
# Compare the value functions for the two types
if IndShockExample.vFuncBool:
    print('Value functions for perfect foresight vs idiosyncratic shocks:')
    plotFuncs([PFexample.solution[0].vFunc,IndShockExample.solution[0].vFunc],
                  IndShockExample.solution[0].mNrmMin+0.5,10)

In [ ]:
# Simulate some data; results stored in mNrmNow_hist, cNrmNow_hist, and pLvlNow_hist
IndShockExample.T_sim = 120
IndShockExample.track_vars = ['mNrmNow','cNrmNow','pLvlNow']
IndShockExample.makeShockHistory() # This is optional, simulation will draw shocks on the fly if it isn't run.
IndShockExample.initializeSim()
IndShockExample.simulate()

## Idiosyncratic shocks consumer with a finite lifecycle

This model represents a consumer with idiosyncratic shocks to permanent and transitory income. Her/is problem is defined by a sequence of income distributions, survival probabilities, and permanent income growth rates, as well as time invariant values for risk aversion, discount factor, the interest rate, the grid of end-of-period assets, and an artificial borrowing constraint.

In [ ]:
# This cell defines a dictionary to make an instance of a "lifecycle idiosyncratic shock" consumer.
LifecycleDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : [0.99,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1],                  # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.01,1.01,1.01,1.01,1.01,1.02,1.02,1.02,1.02,1.02],      # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_retire" : 7,                        # Period of retirement 
    "T_age" : 11,                          # Make sure that old people die at terminal age and don't turn into newborns!
    "T_cycle" : 10,                        # Number of periods in the cycle for this agent type
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [None],                 # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
    
    # Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1,0.2,0.1,0.2,0.1,0.2,0.1,0,0,0],                  # Standard deviation of log permanent shocks to income
    "TranShkStd" : [0.3,0.2,0.1,0.3,0.2,0.1,0.3,0,0,0],                  # Standard deviation of log transitory shocks to income
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    
    # A few other paramaters
    "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                   # Preference shocks currently only compatible with linear cFunc
    "vFuncBool" : False,                    # Whether to calculate the value function during solution          
}

In [ ]:
# Make an idiosyncratic shocks consumer with a finite lifecycle
LifecycleExample = IndShockConsumerType(**LifecycleDict)
LifecycleExample.cycles = 1 # Make this consumer live a sequence of periods -- a lifetime -- exactly once

In [ ]:
# Solve the model
LifecycleExample.solve()
LifecycleExample.unpackcFunc()

In [ ]:
# Plot the consumption functions during working life
print('Consumption functions while working:')
mMin = min([LifecycleExample.solution[t].mNrmMin for t in range(LifecycleExample.T_cycle)])
plotFuncs(LifecycleExample.cFunc[:LifecycleExample.T_retire],mMin,5)

# Plot the consumption functions during retirement
print('Consumption functions while retired:')
plotFuncs(LifecycleExample.cFunc[LifecycleExample.T_retire:],0,5)
LifecycleExample.timeRev()

In [ ]:
# Simulate some data; results stored in mNrmNow_hist, cNrmNow_hist, pLvlNow_hist, and t_age_hist
LifecycleExample.T_sim = 120
LifecycleExample.track_vars = ['mNrmNow','cNrmNow','pLvlNow','t_age']
LifecycleExample.initializeSim()
LifecycleExample.simulate()

## "Cyclical" consumer type 
This model solves a "cyclical" consumer type who lives the same four quarters repeatedly.
The consumer has income that greatly fluctuates throughout the year.

In [ ]:
# This cell defines a dictionary to make an instance of a "cyclical" consumer, 
# an infinite consumer with a four period cycle
CyclicalDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : 4*[0.98],                     # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.082251, 2.8, 0.3, 1.1],                  # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 4,                         # Number of periods in the cycle for this agent type
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [None],                   # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
    
    # Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1,0.1,0.1,0.1],                  # Standard deviation of log permanent shocks to income
    "TranShkStd" : [0.1,0.1,0.1,0.1],                  # Standard deviation of log transitory shocks to income
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    
    # A few other paramaters
    "BoroCnstArt" : 0.0,                   # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                   # Preference shocks currently only compatible with linear cFunc
    "vFuncBool" : False,                    # Whether to calculate the value function during solution          
}

In [ ]:
# Make a "cyclical" consumer type
CyclicalExample = IndShockConsumerType(**CyclicalDict)
CyclicalExample.cycles = 0

In [ ]:
# Solve the model
CyclicalExample.solve()
CyclicalExample.unpackcFunc()

In [ ]:
# Plot the consumption functions for the cyclical consumer type
print('Quarterly consumption functions:')
mMin = min([X.mNrmMin for X in CyclicalExample.solution])
plotFuncs(CyclicalExample.cFunc,mMin,5)

In [ ]:
# Simulate some data; results stored in cHist, mHist, bHist, aHist, MPChist, and pHist
CyclicalExample.T_sim = 480
CyclicalExample.track_vars = ['mNrmNow','cNrmNow','pLvlNow','t_cycle']
CyclicalExample.initializeSim()
CyclicalExample.simulate()

## Agent with a kinky interest rate (Rboro > RSave)

This model represents a consumer that faces idiosyncratic shocks to income and has a different interest factor on saving and on borrowing.  It extends $\texttt{IndShockConsumerType}$, with very small changes.  The solver for this class, $\texttt{solveConsKinkedR}$, is currently only compatible with linear spline interpolation.

In [ ]:
# This cell defines a dictionary to make an instance of a "kinked R" idiosyncratic shock consumer.
KinkedRDict={
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Default intertemporal discount factor
    "LivPrb" : [0.98],                     # Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "PermGroFac" :[1.01],                  # Permanent income growth factor   

    # Parameters for constructing the "assets above minimum" grid
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 20,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [None],                   # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    
    # Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1],                  # Standard deviation of log permanent shocks to income
    "TranShkStd" : [0.1],                  # Standard deviation of log transitory shocks to income
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    
    # A few other paramaters
    "BoroCnstArt" : None,                   # kinked R is a bit silly if borrowing not allowed
    "CubicBool" : False,                   # kinked R currently only compatible with linear cFunc...
    "aXtraCount" : 48,                     # ...so need lots of extra gridpoints to make up for it
    "vFuncBool" : True,                    # Whether to calculate the value function during solution        
    
    'Rboro' : 1.20,           # Interest factor on assets when borrowing, a < 0
    'Rsave' : 1.02,           # Interest factor on assets when saving, a > 0   
}
del KinkedRDict['Rfree'] # get rid of constant interest factor

In [ ]:
# Make an agent with a kinky interest rate
KinkyExample = KinkedRconsumerType(**KinkedRDict)
KinkyExample.cycles = 0 # Make the Example infinite horizon

In [ ]:
# Solve the model
KinkyExample.solve()
KinkyExample.unpackcFunc()

In [ ]:
# Plot the Kinky consumption function
print('Kinky consumption function:')
plotFuncs(KinkyExample.cFunc[0],KinkyExample.solution[0].mNrmMin,5)

In [ ]:
# Simulate some data
KinkyExample.T_sim = 120
KinkyExample.track_vars = ['mNrmNow','cNrmNow','pLvlNow']
KinkyExample.initializeSim()
KinkyExample.simulate()